In [1]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMelbourneV2, FakeMumbaiV2, FakeBrisbane, FakeTorino
#from qiskit_ibm_runtime.options import ZneOptions
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
#distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr

# Setup other constants
symbols  = ['H', 'H']
#gr_energies = []
excited_state_energies = []

    # Define the molecule
#for d in distances:
    
    
geometry = np.array([[0.0, 0.0,  0.0],
                    [0.0, 0.0, 0.735*r]
                    ], requires_grad = False)
active_electrons = 2
active_orbitals = 2
charge = 0


def gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, shots=None, max_iter=100):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    hf_state = qml.qchem.hf_state(active_electrons, qubits)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires=range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    # Define the device
    dev = qml.device("lightning.qubit", wires=qubits, shots=shots)
    
    # Define the qnode
    if shots==None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                 s_wires=s_wires, d_wires=d_wires,
                                                 hf_state=hf_state)

    print(circuit(params, wires, s_wires, d_wires, hf_state))
    return params


def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
    for t in range(10):
        if shots==0:
            #dev = qml.device("lightning.qubit", wires=qubits)
            dev = qml.device("lightning.qubit", wires=qubits)
        else:
            #dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
            for w in occ:
                qml.X(wires=w)
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    #final M matrix
    mat_size = len(list1)
    
    block_size = mat_size/size
    lo_limit = int(rank * block_size)
    up_limit = int((rank+1) * block_size)
    if rank == size-1:
        up_limit = mat_size

    M = np.zeros((mat_size, mat_size))
    M_1d_size = int(mat_size * (mat_size + 1)/2)
    #eig = np.zeros(mat_size)
    Mdiag = np.zeros(mat_size)
    Mdiag_local = np.zeros(mat_size)
    M_local = np.zeros((mat_size, mat_size))
    for i in range(lo_limit, up_limit):
        Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
    comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)

    def idx(n):
        id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
        id_j = n - (id_i-1) * (id_i)/2
        return int(id_i-1), int(id_j-1)

    for ii in range(rank, M_1d_size, size):
        c = ii + 1

        idi, idj = idx(c)

        if idi==idj:
            Mtmp = Mdiag[idi]
        else:
            #print(idi, idj)
            Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

        M_local[idi, idj] = Mtmp
        M_local[idj, idi] = Mtmp

    comm.Reduce(M_local, M, MPI.SUM, 0)
    comm.Bcast(M, root=0)

    eig, evec=np.linalg.eig(M)
    
    return np.sort(eig), evec

params = gs_exact(symbols,geometry, active_electrons, active_orbitals, charge,max_iter=50)

eig, evec = qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0)
#np.save('eigenvalues.npy', eig)
#np.save('eigenvectors.npy', evec)
excited_state_energies.append(eig)
print("qscEOM", excited_state_energies)

-1.1373060354055013


TypeError: qsceom.<locals>.circuit_d() missing 1 required positional argument: 'hf_state'